In [8]:
a = [[0 for x in range(8)] for y in range(8)]
for i in range(8):
    a[0][i] = -1
    a[1][i] = -1
    a[6][i] = 1
    a[7][i] = 1
print(a)




[[-1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]


In [9]:
def offensive_heuristic1(l):
    return len(l)


SyntaxError: invalid syntax (<ipython-input-9-6a56ed5f58bc>, line 2)

In [18]:
def init_pieces():
    wl = set()
    bl = set()
    for i in range(8):
        bl.add((0, i))
        bl.add((1, i))
        wl.add((6, i))
        wl.add((7, i))
    return bl, wl

In [20]:
bl, wl = init_pieces()
#print(bl)

In [24]:
# returns boolean, and the piece eaten
def viable(color, new_pos, straight, bl, wl):
    if color == 1:  # black moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 8 and new_pos[1] < 8:
            # check whether moving onto black piece
            if new_pos in bl:
                return False
            
            # check whether moving onto white piece
            if straight:
                if new_pos in wl:
                    return False
            else: # may eat some piece
                if new_pos in wl:
                    return True, new_pos
                
            return True
        
        else:
            return False
    
    else:           # white moving
        if new_pos[0] > 0 and new_pos[1] > 0 and new_pos[0] < 8 and new_pos[1] < 8:
            # check whether moving onto white piece
            if new_pos in wl:
                return False
            
            # check whether moving onto black piece
            if straight:
                if new_pos in bl:
                    return False
            else: # may eat some piece
                if new_pos in bl:
                    return True, new_pos
                
            return True
        
        else:
            return False

In [25]:
def base_case(bl, wl):
    if (not wl) or (not bl):
        return True
    for p in bl:
        if p[0] == 7:
            return True
    for p in wl:
        if p[0] == 0:
            return True
    
    return False

In [ ]:
# black --> 1
# white --> -1

def maxmove(color, bl, wl, depth):
    
    # base case: all pieces of one agent die   or   one pieces of a agent reach the other side
    if base_case(bl, wl) or depth == 0:
        return offensive_heuristic1(bl, wl)
    
    if color == 1:
        cur_max = 0
        # also record best action
        for p in bl:
            
            # left forward
            new_pos = (p[0]+1, p[1]+1)
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[1])
                    
                val = minmove(-1, new_bl, new_wl, depth-1)
                if (val > cur_max):
                    curmax = val
                    best_action = (-1, new_bl, new_wl)
                    
            # forward
            new_pos = (p[0]+1, p[1])
            via = viable(1, new_pos, True, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                val = minmove(-1, new_bl, new_wl, depth-1)

                
            # right forward
            new_pos = (p[0]+1, p[1]-1)        
            via = viable(1, new_pos, False, bl, wl)
            if via[0]:
                new_bl = bl.copy()
                new_bl.remove(p)
                new_bl.add(new_pos)
                new_wl = wl.copy()
                if via[1]:
                    new_wl.remove(via[1])
                    
                val = minmove(-1, new_bl, new_wl, depth-1)
                if (val > cur_max):
                    curmax = val
                    best_action = (-1, new_bl, new_wl)
                    
           
        # return the most promising result after loop through all pieces
        return cur_max
    else:
        

In [33]:

if True:
    best = 5
print(best)

5


In [ ]:
def minmove():
    